# V3 Fraction Collector (Runze Valve)

In [1]:
from fraction_collector_v3 import FractionCollector
from sample_producer import SampleProducer
import threading
import queue
import time

In [2]:
# Initialize the FractionCollector
collector = FractionCollector(sensor_id=1, runze_valve_port='COM9', runze_valve_address=0, runze_valve_num_port=10, collection_num=3, waste_num=6)

Connected to CNC Machine!
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 2
Moved To (X174, Y20, Z0):  ['ok']


In [3]:
# Define well locations
#loc = 'well_plate_location'
loc = '96_well_plate_location'

In [4]:
def collect_sample_sim(sample, index, duration):
    print(f"🧪 Collecting {sample} at location {index} (takes 15s)...")
    time.sleep(duration)  # Simulate the time taken to collect the sample
    print(f"✅ Finished collecting {sample} at location {index}")

In [ ]:
# Shared queue and event
sample_queue = queue.Queue()
stop_event = threading.Event()
number_of_samples = 4

# Initialize producer
producer = SampleProducer(sample_queue, stop_event, number=number_of_samples, duration=120, delay=200)
# 2 is yellow, 3 is red, 4 is green, 5 is blue
producer.start()

Current port set to 2
Sample 1 is ready to collect!
Current port set to 3
Sample 2 is ready to collect!
Current port set to 4
Sample 3 is ready to collect!
Current port set to 5
Sample 4 is ready to collect!


In [6]:
# Simulate collection
loc_index = 32
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break

        #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
        collection_thread = threading.Thread(target=collector.collect_fraction, args=(10, loc, loc_index, 30)) #real collection
        collection_thread.start()
        loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 2
Moved To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 30 drops...

Sensor reset failed or returned unexpected response: {"ResetSensor":"True"}
Current port set to 3
Waiting for 30 drops...
Drops counted: 3 (poll took 2.082 s)
Drops counted: 4 (poll took 2.074 s)
Drops counted: 5 (poll took 2.059 s)
Drops counted: 7 (poll took 2.060 s)
Drops counted: 8 (poll took 2.076 s)
Drops counted: 9 (poll took 2.078 s)
Drops counted: 10 (poll took 2.084 s)
Drops counted: 12 (poll took 2.076 s)
Drops counted: 13 (poll took 2.077 s)
Drops counted: 14 (poll took 2.068 s)
Drops counted: 16 (poll took 2.077 s)
Drops counted: 17 (poll took 2.077 s)
Drops counted: 18 (poll took 2.078 s)
Drops counted: 19 (poll took 2.074 s)
Drops counted: 21 (poll took 2.085 s)
Drops counted: 22 (poll took 2.076 s)
Drops counted: 23 (poll took 2.076 s)
Drops counted: 25 (poll took 2.064 s)
Drops counted: 26 (po

In [ ]:
stop_event.set()

In [8]:
# Simulation code for sample pin from flow reactor
sample_queue = queue.Queue()
stop_event = threading.Event()

# Producer thread: generates "sample 1", "sample 2", ...
def sample_producer(number=48, duration=120):
    for count in range(1, number+1):
        if stop_event.is_set():
            break
        time.sleep(duration)  # Fixed interval
        sample = f"{count}"
        print(f"Sample {sample} is ready to collect!")
        sample_queue.put(sample)

# Start the background producer thread
producer_thread = threading.Thread(target=sample_producer, kwargs={'number': 6, 'duration': 120}, daemon=True)
producer_thread.start()

Sample 1 is ready to collect!
Sample 2 is ready to collect!
Sample 3 is ready to collect!
Sample 4 is ready to collect!
Sample 5 is ready to collect!
Sample 6 is ready to collect!
